In [ ]:
%pip install wandb huggingface_hub transformers datasets accelerate

In [ ]:
import os
from huggingface_hub import login

os.environ["WANDB_API_KEY"] = userdata.get('wandb')
os.environ["WANDB_NOTEBOOK_NAME"] = "python-code-autocomplete"
os.environ["WANDB_PROJECT"] = "python-code-autocomplete"

login()

In [1]:
import torch
import torch.nn as nn 
import math 

def generate_square_subsequent_mask(sz):
    """Generate a mask to prevent attention to future positions."""
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, ff_hidden_layer, dropout) -> None:
        """
        d_model: The dimension of the input vector.
        num_heads: The number of heads in the multihead attention mechanism.
        ff_hidden_layer: The dimension of the feedforward hidden layer.
        dropout: The dropout rate.
        """
        super(DecoderBlock, self).__init__()
        self.self_attention = nn.MultiheadAttention(d_model, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.linear1 = nn.Linear(d_model, ff_hidden_layer)
        self.linear2 = nn.Linear(ff_hidden_layer, d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, target_mask):
        """
        x: The input tensor.
        target_mask: Masks to prevent the attention to certain positions.
        """
        attn_output, _ = self.self_attention(x, x, x, attn_mask = target_mask)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)
        ff_output = self.linear2(F.relu(self.linear1(x)))
        x = x + self.dropout2(ff_output)
        x = self.norm2(x)
        return x
    
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, ff_hidden_layer, dropout, num_layers):
        super(TransformerDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.transformer_blocks = nn.ModuleList([DecoderBlock(d_model, num_heads, ff_hidden_layer, dropout) for _ in range(num_layers)])
        self.linear = nn.Linear(d_model, vocab_size)
        self.softmax = nn.LogSoftmax(dim = -1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        target_mask = generate_square_subsequent_mask(x.size(0))
        for block in self.transformer_blocks:
            x = block(x, target_mask)
        output = self.linear(x)
        output = self.softmax(output)
        return output


In [ ]:
from transformers import AutoTokenizer

context_legth = 256
tokenizer = AutoTokenizer.from_pretrained("MadMarx37/arxiv-python-research-code-tokenizer")

def tokenize(element):
    outputs = tokenizer(
    element["code"],
    truncation = True,
    max_length = context_legth,
    return_overflowing_tokens = True,
    return_length = True
)
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_legth:
            input_batch.append(input_ids)

    return {
        "input_ids": input_batch
    }



In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Define the hyperparameters
vocab_size     = 10000
d_model        = 2048
num_heads      = 2
ff_hidden_layer  = 8*d_model
dropout        = 0.1
num_layers     = 20
context_length = 1000
batch_size     = 1
# Initialize the model
model = TransformerDecoder(vocab_size, d_model, num_heads, ff_hidden_layer, dropout)


##From GPT2
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
context_length = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384 #384/6 = 64-dimensional head
n_head = 6
n_layer = 6 #6 layers of the above
dropout = 0.2 #prevents overfitting

torch.manual_seed(42)

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("ArtifactAI/arxiv_python_research_code")

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict

ds_train = raw_dataset["train"]
ds_valid = raw_dataset["valid"]

raw_datasets = DatasetDict(
    {
        "train": ds_train.shuffle().select(range(50000)),
        "valid": ds_valid.shuffle().select(range(500))
    }
)

tokenized_datasets = raw_datasets.map(
    tokenize, batched = True, remove_columns = raw_datasets["train"].column_names
)
tokenized_datasets

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir = "python-codecomplete-lm",
    per_device_train_batch_size= 32,
    per_device_eval_batch_size = 32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps = 5_000,
    fp16 = True,
    push_to_hub=True,
    report_to="wandb"
)

trainer = Trainer(
    model = model,
    tokenizer=tokenizer,
    args = args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"]
)

In [ ]:
#train here
trainer.train()
trainer.push_to_hub()

## Eval

In [ ]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model = "MadMarx37/python-codecomplete-lm", device = device
)

In [ ]:
txt = """\
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])